In [ ]:
!pip install PyPDF2 pdfplumber transformers langchain faiss-cpu sentence-transformers
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 827.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 516.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76

In [ ]:
from google.colab import files
import PyPDF2

# get the user to upload their own pdf (this will serve as the knowledge base for applciation)
uploaded = files.upload()

# get the uploaded pdf's filename
for filename in uploaded.keys():
    pdf_path = filename

Saving Electronic_Health_Records_EHR.pdf to Electronic_Health_Records_EHR.pdf


In [ ]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text

# get the text from the pdf
text = extract_text_from_pdf(pdf_path)

# prints the beginning of the pdf so you know that its the correct pdf
print(text[:50])

See discussions, st ats, and author pr ofiles f or


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize

def split_text_into_chunks(text, chunk_size=500):
    sentences = sent_tokenize(text)
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk) + len(sentence) <= chunk_size:
            chunk += sentence + " "
        else:
            chunks.append(chunk.strip())
            chunk = sentence + " "
    if chunk:
        chunks.append(chunk.strip())
    return chunks

# turns the pdf into chuks
chunks = split_text_into_chunks(text)

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

#loads in uses a tool called sentenceTransformer that turns text into numbers so the computer can understand them
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

#turns the chunks into numbers using that transformers model
chunk_embeddings = model.encode(chunks)

#makes a search tool for the computer to use and compare chunks of text that are similar to what the user is looking for when they start chatting with the pdf
index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
index.add(chunk_embeddings)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def retrieve_chunks(question, index, model, chunks, k=5):
    question_embedding = model.encode([question])
    _, indices = index.search(question_embedding, k)
    return [chunks[idx] for idx in indices[0]]


In [ ]:
import openai
#api key(actual key removed for security purposes)
openai.api_key = 'openaiapikeygoeshere:)'

def generate_answer_with_openai(question, context):
    prompt = f"Context: {context}\n\nQuestion: {question}\nAnswer:"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7,
    )
    return response['choices'][0]['message']['content'].strip()

In [ ]:
#chatting function (where the magic happens ;))
def chat_with_pdf(question, index, model, chunks):
    relevant_chunks = retrieve_chunks(question, index, model, chunks)
    context = " ".join(relevant_chunks)
    answer = generate_answer_with_openai(question, context)
    return answer
while True:
    #gets input
    question = input("Ask a question about the PDF (or type 'exit' to stop): ")
    if question.lower() == 'exit':
        print("Exiting chat.")
        break
    #gets response
    response = chat_with_pdf(question, index, model, chunks)
    #outputs response
    print(f"Answer: {response}\n")

Ask a question about the PDF (or type 'exit' to stop): what is this pdf about
Answer: The PDF is about Electronic Health Records (EHR). It discusses how EHRs are electronic versions of patients' healthcare records and the importance of transitioning from paper-based systems to electronic records. The document mentions President Bush's statement about the need for modernizing the healthcare system with electronic medical records and the passing of the American Recovery and Investment Act of 2009 to support this transition.

Ask a question about the PDF (or type 'exit' to stop): Give me the key takeaways from this document
Answer: 1. The document discusses Electronic Health Records (EHR), which are electronic versions of patients' healthcare records.
2. Various sources are cited in the document related to EHR, including articles, websites, and research papers.
3. The importance of efficient crawling through URL ordering is highlighted in the context of EHR.
4. Concerns about a potential 